In [1]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

In [2]:
def get_dataloaders(data_dir, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])

    train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform)
    val_dataset   = datasets.ImageFolder(os.path.join(data_dir, "val"), transform)
    test_dataset  = datasets.ImageFolder(os.path.join(data_dir, "test"), transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader, len(train_dataset.classes)

In [3]:
data_dir = r"D:\STAI_PROJECT\github\Dangerous-Farm-Insects-Classification-main\data\processed\farm_insects\splits" #changable
train_loader, val_loader, test_loader, num_classes = get_dataloaders(data_dir)

## FIXED

In [4]:
# Load InceptionV3
model = models.inception_v3(weights=models.Inception_V3_Weights.IMAGENET1K_V1)
model.aux_logits = False  # Disable auxiliary classifiers for simplicity

# Freeze all parameters
for param in model.parameters():
    param.requires_grad = False

# Replace the final layer
# InceptionV3 fc layer has 2048 input features
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
# Optimize only the final layer
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# --- Training Loop (Same as your original) ---
num_epochs = 10
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    
    for phase, loader in [('train', train_loader), ('val', val_loader)]:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                
                # InceptionV3 might return InceptionOutputs object, we just need the logits
                if hasattr(outputs, 'logits'):
                    outputs = outputs.logits
                
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(loader.dataset)
        epoch_acc = running_corrects.double() / len(loader.dataset)
        print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to C:\Users\dell/.cache\torch\hub\checkpoints\inception_v3_google-0cc3c7bd.pth


100%|██████████| 104M/104M [01:16<00:00, 1.43MB/s] 


Epoch 1/10


c:\ProgramData\anaconda3\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


train Loss: 2.6860 Acc: 0.0971
val Loss: 2.5236 Acc: 0.3162
Epoch 2/10
train Loss: 2.4838 Acc: 0.2735
val Loss: 2.3344 Acc: 0.4743
Epoch 3/10
train Loss: 2.2901 Acc: 0.4172
val Loss: 2.1679 Acc: 0.5296
Epoch 4/10
train Loss: 2.1257 Acc: 0.4975
val Loss: 2.0322 Acc: 0.5889
Epoch 5/10
train Loss: 1.9860 Acc: 0.5322
val Loss: 1.9020 Acc: 0.6126
Epoch 6/10
train Loss: 1.8689 Acc: 0.5580
val Loss: 1.8065 Acc: 0.5968
Epoch 7/10
train Loss: 1.7760 Acc: 0.5867
val Loss: 1.7213 Acc: 0.6403
Epoch 8/10
train Loss: 1.6714 Acc: 0.6165
val Loss: 1.6414 Acc: 0.6561
Epoch 9/10
train Loss: 1.5921 Acc: 0.6700
val Loss: 1.5969 Acc: 0.6285
Epoch 10/10
train Loss: 1.5190 Acc: 0.6551
val Loss: 1.5310 Acc: 0.6561


In [5]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 62.97%


## PARTIAL

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.inception_v3(weights=models.Inception_V3_Weights.IMAGENET1K_V1)
model.aux_logits = False

# 1. Freeze everything first
for param in model.parameters():
    param.requires_grad = False

# 2. Unfreeze the last block (Mixed_7c) and the FC layer
# 'Mixed_7c' is the final inception block before pooling/fc
for param in model.Mixed_7c.parameters():
    param.requires_grad = True

for param in model.fc.parameters():
    param.requires_grad = True

# Replace FC
model.fc = nn.Linear(2048, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
# Pass only trainable parameters to optimizer
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

num_epochs = 20

# --- Training Loop ---
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Training Loss: {avg_loss:.4f}")

# --- Evaluation ---
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("Partial Fine-tuning Accuracy:", accuracy_score(all_labels, all_preds))

Epoch [1/20] - Training Loss: 2.3945
Epoch [2/20] - Training Loss: 1.6283
Epoch [3/20] - Training Loss: 1.1642
Epoch [4/20] - Training Loss: 0.8683
Epoch [5/20] - Training Loss: 0.6495
Epoch [6/20] - Training Loss: 0.4877
Epoch [7/20] - Training Loss: 0.3589
Epoch [8/20] - Training Loss: 0.2600
Epoch [9/20] - Training Loss: 0.1982
Epoch [10/20] - Training Loss: 0.1558
Epoch [11/20] - Training Loss: 0.1298
Epoch [12/20] - Training Loss: 0.1048
Epoch [13/20] - Training Loss: 0.0900
Epoch [14/20] - Training Loss: 0.0782
Epoch [15/20] - Training Loss: 0.0738
Epoch [16/20] - Training Loss: 0.0630
Epoch [17/20] - Training Loss: 0.0563
Epoch [18/20] - Training Loss: 0.0509
Epoch [19/20] - Training Loss: 0.0523
Epoch [20/20] - Training Loss: 0.0445
Partial Fine-tuning Accuracy: 0.7215189873417721


In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# --- Evaluation Code ---
model.eval()  # Critical: Sets dropout and batchnorm layers to evaluation mode
all_preds = []
all_labels = []

print("Starting evaluation on Test set...")

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        
        # Move to CPU to use with sklearn
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# --- Calculate Metrics ---
acc = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
rec = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

print("-" * 30)
print(f"Test Accuracy : {acc*100:.2f}%")
print(f"Precision     : {prec:.4f}")
print(f"Recall        : {rec:.4f}")
print(f"F1 Score      : {f1:.4f}")
print("-" * 30)

Starting evaluation on Test set...


c:\ProgramData\anaconda3\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


------------------------------
Test Accuracy : 72.15%
Precision     : 0.7359
Recall        : 0.7215
F1 Score      : 0.7226
------------------------------


## FULL

In [9]:
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.inception_v3(weights=models.Inception_V3_Weights.IMAGENET1K_V1)
model.aux_logits = False # Disable aux logits to use simple CrossEntropy

# Replace FC
model.fc = nn.Linear(2048, num_classes)
model = model.to(device)

# Unfreeze everything (Default behavior, but good to be explicit if reusing variable)
for param in model.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4) # Low LR for full fine-tuning
num_epochs = 20

print("Starting FULL training...")
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Training Loss: {avg_loss:.4f}")

# --- Evaluation ---
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# --- Metrics ---
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average="weighted", zero_division=0)
recall = recall_score(all_labels, all_preds, average="weighted", zero_division=0)
f1 = f1_score(all_labels, all_preds, average="weighted", zero_division=0)

print("\nFinal Performance Metrics (FULL):")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")

Starting FULL training...


c:\ProgramData\anaconda3\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [1/20] - Training Loss: 2.2426
Epoch [2/20] - Training Loss: 1.1603
Epoch [3/20] - Training Loss: 0.5805
Epoch [4/20] - Training Loss: 0.2985
Epoch [5/20] - Training Loss: 0.1708
Epoch [6/20] - Training Loss: 0.1035
Epoch [7/20] - Training Loss: 0.0718
Epoch [8/20] - Training Loss: 0.0609
Epoch [9/20] - Training Loss: 0.0537
Epoch [10/20] - Training Loss: 0.0444
Epoch [11/20] - Training Loss: 0.0395
Epoch [12/20] - Training Loss: 0.0435
Epoch [13/20] - Training Loss: 0.0365
Epoch [14/20] - Training Loss: 0.0337
Epoch [15/20] - Training Loss: 0.0300
Epoch [16/20] - Training Loss: 0.0276
Epoch [17/20] - Training Loss: 0.0308
Epoch [18/20] - Training Loss: 0.0262
Epoch [19/20] - Training Loss: 0.0257
Epoch [20/20] - Training Loss: 0.0245

Final Performance Metrics (FULL):
Accuracy : 0.7468
Precision: 0.7542
Recall   : 0.7468
F1 Score : 0.7466
